We are going to implement a sudoku solver.
This is inspired by the solution by Peter Norvig (http://norvig.com/sudoku.html)

It utilisees constrait satisfaction and propogation as well as search algorithm to implement the solution.

This code also contains a generator to produce a random sudoku problem which can then be solved by the solver.

## Terminology used

*rows: Represented by alphabets from A to I  
*columns: Represented by numbers from 1 to 9  
*cells: Holds label of each square that can carry a value from 1 to 9. Labeling is a combination of row and column value. ie., A1, E7  
*grid: The string representing the given sudoku problem with . or 0 for the empty cells  
*units_list: List of all the units in the Sudoku. ie., the rows, columns and 3*3 squares   
*units: Dictionary with cell label as key and list of labels of units that contain the key cell  
*peers: Dictionary with cell label as key and list of labels of all the cells that can contradict the given cell  
*values: Dictionary with cell label as key and list of possible values of the cell

In [1]:
def gen(A,B):       #generates the name of the cell, eg., 'A1' is first cell and 'I9' is last cell
    return [a+b for a in A for b in B]

digits='123456789'
rows='ABCDEFGHI'
cols=digits
cells=gen(rows,cols)


#The dimensions that require to be checked for each cell are in the units. These include the row, column and 3*3 square containing the cell
units_list=([gen(rows,c) for c in cols] +
          [gen(r,cols) for r in rows] +
          [gen(r,c) for r in ('ABC','DEF','GHI') for c in ('123','456','789')])

units=dict((c,[u for u in units_list if c in u]) for c in cells)


#All the cells that can contradict a cell are the peers of that cell
peers = dict((c, set(sum(units[c],[]))-set([c]))
             for c in cells)

In [2]:
print(peers['A1'])

{'A8', 'A3', 'B3', 'A5', 'F1', 'B2', 'A7', 'A9', 'G1', 'A2', 'C3', 'C1', 'I1', 'E1', 'A6', 'C2', 'B1', 'A4', 'H1', 'D1'}


In [3]:
#Convert the list of the values into a dictionary with given cell as key

def parse_grid(grid):
    values=dict((c, digits) for c in cells)
    for c,d in grid_values(grid).items():
        if d in digits and not assign(values,c,d):
            return False
    return values

def grid_values(grid):
    given = [g for g in grid if g in digits or g in '0.']
    assert len(given) == 81
    return dict(zip(cells, given))

In [4]:
def display(values):
    width = 1+max(len(values[c]) for c in cells)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': 
            print(line)
    print

In [5]:
#To assign a value to a cell, all other values of the cell are eliminated

def assign(values,c,d):
    other_values=values[c].replace(d,'')
    if all(eliminate(values,c,d2) for d2 in other_values):
        return values
    else:
        return False

In [6]:
#To eliminate a value from a cell, first remove value from values[cell].

def eliminate(values,c,d):
    if d not in values[c]:
        return values
    
    values[c]=values[c].replace(d,'')
    if len(values[c])==0:
        return False
    elif len(values[c])==1:
        d2=values[c]
        if not all(eliminate(values, c2, d2) for c2 in peers[c]):
            return False
    
#Then, check each unit of the cell that has given digit in values. If there's only one possible cell, assign value to that cell
    for u in units[c]:
        d_cells=[c for c in u if d in values[c]]        
        if len(d_cells)==0:
            return False
        elif len(d_cells)==1:
            if not assign(values,d_cells[0],d):
                return False    
    return values

In [7]:
#Return some element of seq that is true. This is used to give false value to the branches that do not give solution
def some(seq):
    for e in seq:
        if e:
            return e
    return False

In [8]:
#This calls the search function
def solve(grid):
    return search(parse_grid(grid))


#Using depth-first search and propagation, try all possible values.
def search(values):
    if values is False:
        return False     # Failed earlier
    
    if all(len(values[c]) == 1 for c in cells): 
        return values    # Solved!

    # Chose the unfilled cell c with the fewest possibilities
    n,c = min((len(values[c]),c) for c in cells if len(values[c]) > 1)
    return some(search(assign(values.copy(), c, d)) for d in values[c])

In [9]:
#Return a randomly shuffled copy of the input sequence. This is used to select which cells must be filled to geneate sudoku
def shuffled(seq):
    seq = list(seq)
    random.shuffle(seq)
    return seq

In [10]:
#Create a random sudoku puzzle with N or more values given. When contradiction occurs, restart the process
def random_puzzle(N=17):
    values = dict((c, digits) for c in cells)
    for c in shuffled(cells):
        if not assign(values, c, random.choice(values[c])):
            break
        dc = [values[c] for c in cells if len(values[c]) == 1]
        if len(dc) >= N and len(set(dc)) >= 8:
            return ''.join(values[c] if len(values[c])==1 else '.' for c in cells)   #puzzle in the grid form
    return random_puzzle(N)         # Give up and make a new puzzle

In [11]:
import time, random

#Calls the solve function and provides all outputs
def solution(grid):
    start = time.process_time()
    values = solve(grid)
    t = time.process_time()-start

    print('\n\tThe generated Sudoku: ')
    display(grid_values(grid))
    
    print('\n\n\tSolved Sudoku: ')
    if values:     #If solution found
        display(values)
    print('\n\tTime taken: (%.2f seconds)\n' %t)


In [12]:
#Implement solution on a randomly generated puzzle.
N=int(input('Enter the minimum number of values that must be already given in the sudoku puzzle: '))
solution(random_puzzle(N))

Enter the minimum number of values that must be already given in the sudoku puzzle: 10

	The generated Sudoku: 
. 5 3 |. . . |. 7 . 
8 . 9 |. . . |. 3 2 
. . 7 |. 3 . |. . 9 
------+------+------
9 . . |. 2 . |3 . . 
. . 2 |. . . |. . . 
. . . |9 1 . |2 . . 
------+------+------
. 9 . |. . . |. . . 
. . . |. 4 . |. . . 
4 . . |. . . |. 2 1 


	Solved Sudoku: 
1 5 3 |2 6 9 |4 7 8 
8 4 9 |1 5 7 |6 3 2 
2 6 7 |4 3 8 |1 5 9 
------+------+------
9 7 4 |8 2 5 |3 1 6 
6 1 2 |3 7 4 |8 9 5 
5 3 8 |9 1 6 |2 4 7 
------+------+------
3 9 1 |7 8 2 |5 6 4 
7 2 5 |6 4 1 |9 8 3 
4 8 6 |5 9 3 |7 2 1 

	Time taken: (0.02 seconds)

